<a href="https://www.kaggle.com/code/ismailmohameddarwish/exploring-european-soccer-database?scriptVersionId=103566703" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Expotatory data analysis #

 In this notebook i will  use the **European Soccer Database** published by **HUGO MATHIEN** in this [link](https://www.kaggle.com/datasets/hugomathien/soccer) to perform some analysis using SQL in extracting useful queries and use python to explore this data and answer some question
 
 ## About database ##
 this data base include the data fot 10 european leaguess from 2009-2016 including matches,teams and players
 it is almost clean data and ready for analysis
 
 

In [1]:
#importing used libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import sqlite3 as sql

# importing data and creating connction to the database to read sqlite queries

path='../input/soccer/database.sqlite'
con=sql.connect(path)

#now we can write sql queries
# explore different tables in the database

sql_master_tables = pd.read_sql('select * from sqlite_master where type="table";',con)
sql_master_tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


So we have 7 tables in this database (as "sqlite_sequence" table is  created and initialized automatically whenever a normal table that contains an AUTOINCREMENT column is created )
so we ignore it

In [2]:
# now listing  tables name and dropping "sqlite_sequence" table

tablesName = pd.read_sql('select name from sqlite_master where type="table";',con)  #quering tables name
tablesName=tablesName[1:8].squeeze()       #convert the resulting df to series
tablesName=tablesName.values.tolist()      # make a list of tables name
tablesName



['Player_Attributes',
 'Player',
 'Match',
 'League',
 'Country',
 'Team',
 'Team_Attributes']

In [3]:
# now we can explore each table and ask some question


### what the premier league standing in season 2015/2016

In [4]:
league = pd.read_sql('''
SELECT home.team_long_name AS homeTeam,
       match.home_team_goal,
       (CASE
            WHEN home_team_goal = away_team_goal THEN 1
            WHEN home_team_goal > away_team_goal THEN 3
            WHEN home_team_goal < away_team_goal THEN 0
        END)pointHome,
       away.team_long_name AS awayTeam,
       Match.away_team_goal,
       (CASE
            WHEN home_team_goal = away_team_goal THEN 1
            WHEN home_team_goal > away_team_goal THEN 0
            WHEN home_team_goal < away_team_goal THEN 3
        END)pointAway
FROM Country
JOIN MATCH ON match.country_id = country.id
AND name = "England"
AND season = "2015/2016"
LEFT JOIN Team AS home ON home.team_api_id = Match.home_team_api_id
LEFT JOIN Team AS away ON away.team_api_id = Match.away_team_api_id
;''', con)

league


,homeTeam,home_team_goal,pointHome,awayTeam,away_team_goal,pointAway
0,Arsenal,0,0,West Ham United,2,3
1,Bournemouth,0,0,Aston Villa,1,3
2,Chelsea,2,1,Swansea City,2,1
3,Everton,2,1,Watford,2,1
4,Leicester City,4,3,Sunderland,2,0
...,...,...,...,...,...,...
375,Southampton,2,1,Leicester City,2,1
376,Swansea City,0,0,Stoke City,1,3
377,Tottenham Hotspur,0,1,Liverpool,0,1
378,Watford,0,0,Arsenal,3,3


In [5]:

league1 = league[['homeTeam', 'home_team_goal', 'pointHome']]

league2 = league[['awayTeam', 'away_team_goal', 'pointAway']]


totalleague = pd.DataFrame(np.concatenate((
    league1.values, league2.values), axis=0))
totalleague.columns = ['team', 'goals', 'points']


leagueSummary = totalleague.groupby('team')[['goals', 'points']].sum()
standing= leagueSummary.sort_values(by='points', ascending=False)

standing

,goals,points
team,,
Leicester City,68,81
Arsenal,65,71
Tottenham Hotspur,69,70
Manchester United,49,66
Manchester City,71,66
Southampton,59,63
West Ham United,65,62
Liverpool,63,60
Stoke City,41,51


### I can validate my result from this[link](https://www.google.com/search?q=pl+standings+15%2F16&oq=pl+sta&aqs=chrome.0.69i59j69i57j0i271.10502j0j7&sourceid=chrome&ie=UTF-8#sie=lg;/m/012vp1qy;2;/m/02_tc;st;fp;1;;;)




## No. of goals per top 5 leagues ##

In [6]:
pd.read_sql("select * from country", con)

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain
